In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re
import time

In [2]:
list_topic=['politics','money','culture','society','world']
list_subtopic=[['assemgov','bluehouse','diplomacy','nk','northkorea'],
               ['indusrty','finance','science','globaleconomy','labor'],
               ['book','perfomance','art','classic','song','broadcast','movie'],
              ['law','education','accident','welfare','traffic','environment','region','health'],
              ['northame','china','japan','eu','etc']]
topic_dict={'politics':"정치",'money':"경제",'culture':"라이프",'society':"사회",'world':"국제"}

In [3]:
import pandas as pd
import numpy as np
#기간설정
dt_index = pd.date_range(start='20200101', end='20200103')

In [4]:
str(dt_index.date[0])

'2020-01-01'

In [5]:
# 각각의 리스트 생성
topic_list=[]
date_list=[]
title_list=[]
text_list=[]

In [6]:
def no_space(text):
    text1=re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
    text2=re.sub('\n\n','',text1)
    return text2

In [7]:
#크롤링 def
def get_news(date,main_url,sub_url,topic,sub_topic):
    html = urlopen(main_url+date)
    bsObject = BeautifulSoup(html, "html.parser")
    
    for meta in bsObject.select('#content span.thumb a'):
        link=meta['href']
        date=date
        url=urlopen(sub_url+link)
        bs_obj=BeautifulSoup(url, "html.parser")
        title=bs_obj.select_one("title").text.split('-')[0]
        x=bs_obj.select_one("#article_body").text
        title=no_space(title)
        text=no_space(x)
        date_list.append(date)
        topic_list.append(topic_dict[topic])
        title_list.append(title)
        text_list.append(text)

    if bsObject.select('#content a.link_page')!=[]:
        #재귀 호출
        url_new="https://news.joins.com/"+topic+"/"+sub_topic+"list/2?Date="
        return get_news(date,url_new,sub_url,topic,sub_topic)
    else:
        return bsObject

In [8]:
#크롤링 시작
num=0
for num_main in tqdm(range(len(list_topic)),desc='Topic_Progress',leave=False):
    for num_sub in tqdm(range(len(list_subtopic[num])),desc='Sub_Topic_Progress',leave=False):
        main_url="https://news.joins.com/"+list_topic[num_main]+"/"+list_subtopic[num_main][num_sub]+"/list/1?filter=OnlyJoongang&date="
        sub_url='https://news.joins.com/article/'
        for num_date in tqdm(range(len(dt_index.date)),desc='Day_Progress'):
            date=str(dt_index.date[num_date])
            bs=get_news(date,main_url,sub_url,list_topic[num_main],list_subtopic[num][num_sub])
            time.sleep(0.01)
            
            
    num+=1


In [9]:
#판다스로 저장 (후에 csv파일로 저장 라인 추가)

df=pd.DataFrame({"date":date_list, "title":title_list, "topic":topic_list,"text":text_list}) 
print(df) 

           date                                         title topic  \
0    2020-01-01          황교안 “전광훈, 나라사랑 마음 커…종교인 사법 제재 신중해야”     정치   
1    2020-01-01            백혜련 “수사‧기소권 전면 분리시 공수처 구조도 달라질 것”     정치   
2    2020-01-01  [포토사오정]민주당 영입 1·2호의 신년 메시지, 키워드는 "희망"과 "소외"     정치   
3    2020-01-01         여야, 2020년 첫날 현충원 방명록에 남긴 글로 본 새해 다짐은     정치   
4    2020-01-01    민주당 “공수처 통과 3표밖에 여유 없다”…당론에 밀린 조응천·금태섭 소신     정치   
..          ...                                           ...   ...   
460  2020-01-03            '이란 군부실세' 솔레이마니 쿠드스 사령관, 미군공습에 사망     국제   
461  2020-01-03        '습격'에 중동으로 눈 돌린 美···"北, 관심 끌려 도발 가능성"     국제   
462  2020-01-03          곤 전 회장, AFP에 편지 “레바논 도피에 아내 역할 없었다”     국제   
463  2020-01-03                           [사진] 미 신속대응군 중동 파병     국제   
464  2020-01-03             13명 탑승한 블랙호크 추락…대만군 참모총장 등 8명 사망     국제   

                                                  text  
0    \n     황교안 자유한국당 대표. [연합뉴스]               자유한국...  
1    \n     백혜련 더불어민주당 의원. [연합뉴스]